In [ ]:
%matplotlib notebook
import rospy
import actionlib
import actionlib.msg
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.widgets import Button
import threading
from std_srvs.srv import *
import time
import sys
import select
import os
import assignment_2_2022.msg
# Import my custom message defined in the /msg folder inside the 'assignment_2_2022' package
from assignment_2_2022.msg import My_pos_vel, PlanningActionGoal, PlanningAction, PlanningGoal
# Import my custom service defined in the /srv folder inside the 'assignment_2_2022' package
from assignment_2_2022.srv import Goals_number, Goals_numberRequest, Goals_numberResponse

# Initialize ROS node
rospy.init_node('robot_controller')

# Initialize the x and y data lists
x_data = []
y_data = []

# Initialize the reached and not reached targets counters
reached_targets = 0
not_reached_targets = 0
obstacle_distance = 0.0

# Create the figure and axis objects
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

# Create the line object for the robot position plot
ln, = ax1.plot([], [], 'ro')
# Set the plot limits
ax1.set_xlim(-10, 10)
ax1.set_ylim(-10, 10)

# Create the bar plot for the number of reached/not-reached targets
targets = ['Reached', 'Not Reached']
target_counts = [reached_targets, not_reached_targets]
bar_plot = ax2.bar(targets, target_counts)
ax2.set_ylim(0,5)

# Create GUI widgets
robot_position_text = widgets.Text(description='Robot position:', value='(0, 0)')
obstacle_distance_text = widgets.Text(description='Closest obstacle:', value='Infinity')

def laser_callback(laser_scan):
    # Find closest obstacle
    min_range = min(laser_scan.ranges)
    if min_range == float('inf'):
        obstacle_distance = 'Infinity'
    else:
        obstacle_distance = '{:.1f}'.format(min_range)
    # Update GUI
    obstacle_distance_text.value = obstacle_distance

# Update the robot position plot and the text boxes
def update_plot(frame):
    ln.set_data(x_data, y_data)
    global target_counts
    target_counts = [reached_targets, not_reached_targets]
    for rect, count in zip(bar_plot.patches, target_counts):
        rect.set_height(count)
    fig.canvas.draw()
    return ln,

def position_callback(msg):
    global x_data, y_data
    # Retrieve the actual position
    x = msg.x
    y = msg.y
    # Update GUI
    robot_position_text.value = '({:.2f}, {:.2f})'.format(x, y)
    # Update the x and y data lists
    x_data.append(x)
    y_data.append(y)

def callback_result(msg):
    status = msg.status.status
    # If status = 3, the goal is reached
    if status == 3:
        increase_reached_targets()
    # If status = 2, the goal is cancelled
    elif status == 2:
        increase_not_reached_targets()

#Callback function for the subscriber to the /reached_targets topic
def increase_reached_targets():
    global target_counts
    global reached_targets
    reached_targets += 1
    #update_bar_plot()

#Callback function for the subscriber to the /not_reached_targets topic
def increase_not_reached_targets():
    global target_counts
    global not_reached_targets
    not_reached_targets += 1
    #update_bar_plot()
    
def goal_callback(msg):
    # Process the received goal message
    x = msg.goal.target_pose.pose.position.x
    y = msg.goal.target_pose.pose.position.y
    # Plot the target
    ax1.plot(x, y, 'bo') 

#Update the GUI layout
gui_layout = widgets.VBox([robot_position_text, obstacle_distance_text])

#Display the GUI
display(gui_layout)

#Create the animation
ani = FuncAnimation(fig, update_plot, interval=100)
  
#Create the subscribers for the relevant topics
#rospy.Subscriber('/odom', Odometry, callback_odom)
rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, callback_result)
rospy.Subscriber('/scan', LaserScan, laser_callback)
rospy.Subscriber("/my_pos_vel", My_pos_vel, position_callback)
rospy.Subscriber('/reaching_goal/goal', PlanningActionGoal, goal_callback)
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

#Start the event loop
plt.show()


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [ ]:
goal = PlanningGoal()

# Define callback function for target set button
def set_callback(sender):
    global x_goal, y_goal
    # Get x and y coordinates from text boxes
    x_goal = float(x_textbox.value)
    y_goal = float(y_textbox.value)
        
    goal.target_pose.pose.position.x = x_goal
    goal.target_pose.pose.position.y = y_goal

    client.send_goal(goal)

def cancel_callback(bot):
    global client
    client.wait_for_server()
    client.cancel_goal()
    
# Create text boxes for target coordinates
x_textbox = widgets.FloatText(description='X:')
y_textbox = widgets.FloatText(description='Y:')

# Create buttons to set and cancel targets
target_set_button = widgets.Button(description='Set Target')
target_cancel_button = widgets.Button(description='Cancel Target')

# Attach callback functions to buttons
target_set_button.on_click(set_callback)
target_cancel_button.on_click(cancel_callback)

# Display widgets
display(widgets.HBox([x_textbox, y_textbox, target_set_button, target_cancel_button]))
